In [ ]:
from utils import load_experiment_results_with_pickle, plot_episode_returns, plot_report_card_metric, calculate_mean_std_of_metric_per_algorithm
import maze_library


In [ ]:
results = load_experiment_results_with_pickle("../mazeLogs/14-05_13-07PPO/Run0/experiment_results.pkl")
training_report_parsed = [maze_library.report_card_from_json(rc) for rc in results["training_report"]]


testing_maze_report_parsed = {
    k: [maze_library.report_card_from_json(rc) for rc in v]
    for k, v in results["testing_maze_report_cards"].items()
}

plot_episode_returns(results["training_explore_rewards"], results["training_exploit_rewards"], 10)

In [ ]:
print(results["params"]["gen_algorithm"])
plot_report_card_metric(testing_maze_report_parsed, "total_percentage_explored")

## Calculate Required Metrics

In [ ]:
algorithms = list(testing_maze_report_parsed.keys())
agent_score = []

# Calculate average_run_score per algorithm (ignoring zeros)
for algo in algorithms:
    values = [getattr(report, "average_run_score") for report in testing_maze_report_parsed[algo] if getattr(report, "average_run_score") != 0]
    avg_score = sum(values) / len(values) if values else 0.0
    agent_score.append(avg_score)

# Get other metrics
algorithms, dijkstra_score, _ = calculate_mean_std_of_metric_per_algorithm(testing_maze_report_parsed, "dijkstra_shortest_path_score")
_, success_rate, _ = calculate_mean_std_of_metric_per_algorithm(testing_maze_report_parsed, "success_rate_in_exploitation")

# Print all variables separately
print("\n=== Experiment Parameters ===")
print(f"Generation Algorithm: {results['params']['gen_algorithm']}")
compund_scores = []
print("\n=== Exploitation Metrics per Algorithm ===")
for i, algo in enumerate(algorithms[:5]):
    # print(f"  Agent Score: {agent_score[i]:.4f}")
    # print(f"  Dijkstra Score: {dijkstra_score[i]:.4f}")
    # print(f"  Success Rate: {success_rate[i]:.4f}")
    if agent_score[i] != 0:
        compound_score = (dijkstra_score[i] / agent_score[i]) * success_rate[i]
        # print(f"  Compound Score: {compound_score:.4f}")
        print("\\hline")
        print(f"{algo} & {dijkstra_score[i]:.2f} & {agent_score[i]:.2f} & {success_rate[i]:.2f} & {compound_score:.2f} \\\\")
        compund_scores.append(compound_score)
    else:
        print("  Compound Score: Undefined (agent_score is 0)")
total_avg_agent_score = sum(agent_score) / len(agent_score) if agent_score else 0.0
total_avg_dijk_score = sum(dijkstra_score) / len(dijkstra_score) if dijkstra_score else 0.0
success_rate_score = sum(success_rate) / len(success_rate) if success_rate else 0.0
compound_score_avg = sum(compund_scores) / len(compund_scores) if compund_scores else 0.0
print("\\hline")

print(f"\\textbf{{Overall Average}} & \\textbf{{{total_avg_dijk_score:.2f}}} & \\textbf{{{total_avg_agent_score:.2f}}} & \\textbf{{{success_rate_score:.2f}}} & \\textbf{{{compound_score_avg:.2f}}} \\\\")
print("\\hline")


In [ ]:
algorithms = list(testing_maze_report_parsed.keys())

# Get other metrics
_, exploration_percentage, _ = calculate_mean_std_of_metric_per_algorithm(testing_maze_report_parsed, "total_percentage_explored")
_, local_tile_revisits, _ = calculate_mean_std_of_metric_per_algorithm(testing_maze_report_parsed, "average_visits")
_, local_exploration_percentage, _ = calculate_mean_std_of_metric_per_algorithm(testing_maze_report_parsed, "percentage_visited")

# Print experiment parameters
print("\n=== Experiment Parameters ===")
print(f"Generation Algorithm: {results['params']['gen_algorithm']}")
compound_scores = []
# Print metrics per algorithm in compact format
print("\n=== Exploration Metrics per Algorithm ===")
for i, algo in enumerate(algorithms[:5]):
    print("\\hline")
    
    compound_score = ((local_exploration_percentage[i] * 100) / local_tile_revisits[i])
    compound_scores.append(compound_score)
    print(f"{algo} & {exploration_percentage[i]:.2f} & {local_tile_revisits[i]:.2f} & {local_exploration_percentage[i]:.2f} & {compound_score:.2f} \\\\")
avg_exploration_percentage = sum(exploration_percentage) / len(exploration_percentage) if exploration_percentage else 0.0
avg_local_tile_revisits = sum(local_tile_revisits) / len(local_tile_revisits) if local_tile_revisits else 0.0
avg_local_exploration_percentage = sum(local_exploration_percentage) / len(local_exploration_percentage) if local_exploration_percentage else 0.0
compound_score_avg = sum(compound_scores) / len(compound_scores) if compound_scores else 0.0
print("\\hline")
print(f"\\textbf{{Overall Average}} & \\textbf{{{avg_exploration_percentage:.2f}\\%}} & \\textbf{{{avg_local_tile_revisits:.2f}}} & \\textbf{{{avg_local_exploration_percentage:.2f}\\%}} & \\textbf{{{compound_score_avg:.2f}}} \\\\")
print("\\hline")


In [ ]:
algorithms = list(testing_maze_report_parsed.keys())

# Get other metrics
_, walls_hit, _ = calculate_mean_std_of_metric_per_algorithm(testing_maze_report_parsed, "walls_hit")
_, full_turns_done, _ = calculate_mean_std_of_metric_per_algorithm(testing_maze_report_parsed, "full_turns_done")

# Print experiment parameters
print("\n=== Experiment Parameters ===")
print(f"Generation Algorithm: {results['params']['gen_algorithm']}")

# Print metrics per algorithm in compact format
print("\n=== Inefficiency Metrics per Algorithm ===")
for i, algo in enumerate(algorithms[:5]):
    compound_score = walls_hit[i] + full_turns_done[i]
    print("\\hline")
    
    print(f"{algo} & {walls_hit[i]:.2f} & {full_turns_done[i]:.2f} & {compound_score:.2f} \\\\")
avg_walls_hit = sum(walls_hit) / len(walls_hit) if walls_hit else 0.0
avg_full_turns_done = sum(full_turns_done) / len(full_turns_done) if full_turns_done else 0.0
print("\\hline")
print(f"\\textbf{{Overall Average}} & \\textbf{{{avg_walls_hit:.2f}}} & \\textbf{{{avg_full_turns_done:.2f}}} & \\textbf{{{(avg_walls_hit + avg_full_turns_done):.2f}}} \\\\")
print("\\hline")